In [13]:
import torch
import numpy as np

In [5]:
a = torch.tensor([[0,0], [1, 1], [0, 0], [0, 0], [0, 0], [0, 0]])
unique = torch.unique(a, dim=0)
H = torch.all(a[:, None] == unique, dim=-1)
w = torch.tensor([0.8, 0.1, 0.1, 0.01])
doubles = torch.where(abs(torch.sum(a[:, None] - unique, dim=-1)) == 2)
print(abs(torch.sum(a[:, None] - unique, dim=-1)) == 2)
doubles

tensor([[False,  True],
        [ True, False],
        [False,  True],
        [False,  True],
        [False,  True],
        [False,  True]])


(tensor([0, 1, 2, 3, 4, 5]), tensor([1, 0, 1, 1, 1, 1]))

In [6]:
def lipkin_local(dist_s, unique_s, weight_s, eps, V, W):
    size = weight_s[torch.where(torch.all(dist_s[:, None] == unique_s, dim=-1))[1]]
    H_0 = eps*(torch.sum(dist_s == 1, dim=1) - torch.sum(dist_s == 0, dim=1))
    size_mask, weight_mask = torch.where(torch.abs(torch.sum(dist_s[:, None]-unique_s, dim=-1)) == 2)
    H_1 = torch.zeros_like(size)
    H_1[size_mask] = V*weight_s[weight_mask]
    sum = 0.5*H_0 + H_1
    return sum

lipkin_local(a, unique, w, 1, 1, 0)

tensor([-0.9000,  1.8000, -0.9000, -0.9000, -0.9000, -0.9000])

In [7]:
import itertools
lst = list(itertools.product([0, 1], repeat=3))
lst

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [8]:
dist_s = torch.tensor([[0, 0]]*24 + [[1, 1]]*4)
size = dist_s.shape[0]
unique, weight = torch.unique(dist_s, dim=0, return_counts=True)
weight = torch.sqrt(weight/size)
mask_u = torch.where(torch.all(dist_s[:, None] == unique, dim = -1))[1]

H_0 = 0.5*(torch.sum(unique == 1, dim=-1) - torch.sum(unique == 0, dim = -1))
H_1 = torch.sum(weight[:, None]*(abs(torch.sum((unique[:, None] - unique), dim=-1)) == 2), dim=0)/weight
E = (H_0[mask_u] - H_1[mask_u])
torch.mean(E)

tensor(-1.4141)

In [9]:
d = torch.tensor([[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]])
a = torch.tensor([[1, 1], [1, 0], [0, 1], [0, 0]])
b = torch.tensor([2, 2, 2])

a[:, :, None]@d[:, None, :]



tensor([[[1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 1]],

        [[0, 0, 0],
         [0, 0, 0]]])

In [10]:
def ising_true(N, J, L):
    hi = nk.hilbert.Spin(s=1/2, N=N) 
    H = sum([L*sigmax(hi,i) for i in range(N)])
    H += sum([J*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
    sp_h=H.to_sparse()
    from scipy.sparse.linalg import eigsh
    eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
    E_gs = eig_vals[0]
    return E_gs

In [11]:
dist_s = torch.tensor([[1, 1], [1, 0], [0, 1], [1, 0], [0, 1], [0,0], [0, 0], [0, 1], [0, 0], [0, 0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [1,1], [1,1], [1,1], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0]])
size = dist_s.shape[0]
unique, weight = torch.unique(dist_s, dim=0, return_counts=True)
weight = torch.sqrt(weight/size)
mask_u = torch.where(torch.all(dist_s[:, None] == unique, dim = -1))[1]

unique = 2*unique-1
H = -torch.sum(torch.roll(unique, 1, dims=-1)*unique, dim = -1)
H

tensor([-2,  2,  2, -2])

In [24]:
def lipkin_local(eps, V, W, samples):

    size = samples.shape[0]
    unique, weight = torch.unique(samples, dim=0, return_counts=True)
    weight = torch.sqrt(weight/size)
    mask = torch.where(torch.all(samples[:, None] == unique, dim = -1))[1]
    
    N_0 = torch.sum(unique == 0, dim=-1)
    N_1 = torch.sum(unique == 1, dim=-1)

    diff_unique = abs(torch.sum(unique[:, None] - unique, dim=-1))
    diff_N1 = abs(N_1[:, None] - N_1)

    H_0 = 0.5*eps*(N_1-N_0)
    H_eps = H_0[mask]
    
    one_pair = torch.bitwise_and(diff_unique==2, diff_N1==2)
    H_1 = V*torch.sum(weight[:, None]*one_pair, dim=0)/weight
    H_V = H_1[mask]
    
    H_2 = W*torch.sum(weight[:, None]*(diff_unique == 1), dim=0)/weight
    H_W = H_2[mask]
    
    E = (H_eps - H_V - H_W)
    return E

dist_s = torch.tensor([[1, 1]]*6+[[0,0]]*35)
print(lipkin_local(1,1,0,dist_s))
torch.mean(lipkin_local(1,1,0,dist_s))

tensor([-1.4152, -1.4152, -1.4152, -1.4152, -1.4152, -1.4152, -1.4140, -1.4140,
        -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140,
        -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140,
        -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140,
        -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140, -1.4140,
        -1.4140])


tensor(-1.4142)

In [20]:
eps = 1; V = 1
H = torch.tensor([[eps, 0, -V], [0, 0, 0], [-V, 0, -eps]], dtype=torch.float64)
eigvals = torch.real(torch.linalg.eigvals(H))
min_eigval = torch.min(eigvals)
print(min_eigval.item())

-1.4142135623730951


In [27]:
eps = 1; V = 1; W = 0
H = torch.tensor([
    [-2*eps, 0, np.sqrt(6)*V, 0, 0],
    [0, -eps + 3*W, 0, 3*V, 0],
    [np.sqrt(6)*V, 0, 4*W, 0, np.sqrt(6)*V],
    [0, 3*V, 0, eps+3*W, 0],
    [0, 0, np.sqrt(6)*V, 0, 2*eps]
])
eigvals = torch.real(torch.linalg.eigvals(H))
min_eigval = torch.min(eigvals)
print(eigvals)

tensor([-4.0000e+00, -2.1218e-16,  4.0000e+00,  3.1623e+00, -3.1623e+00],
       dtype=torch.float64)


In [30]:
def dd(a, b):
    if a==b:
        return 1
    else:
        return 0

def construct_spin(S):

    size = int(2*S + 1)
    J_pluss = np.zeros((size, size))
    J_minus = np.zeros((size, size))
    J_z = np.zeros((size, size))

    for i in range(size):
        for k in range(size):
            m = i-S
            n = k-S

            pm_factor = np.sqrt(S*(S+1) -m*n)

            J_pluss[i, k] = dd(m, n+1)*pm_factor
            J_minus[i, k] = dd(m+1, n)*pm_factor
            J_z[i, k] = dd(m,n)*m
    
    return J_z, J_pluss, J_minus

def construct_Hamiltonian(S, eps, V):
    
    J_z, J_pluss, J_minus = construct_spin(S)

    H = eps*J_z - 1/2*V*(J_pluss@J_pluss + J_minus@J_minus)

    return H

In [35]:
H = torch.tensor(construct_Hamiltonian(2, 1, 0.5))
eigvals = torch.real(torch.linalg.eigvals(H))
min_eigval = torch.min(eigvals)
print(min_eigval.item())

-2.645751311064594
